In [16]:
pip install unidecode scikit-learn BeautifulSoup4 pandas requests lxml selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 29.9 kB/s eta 0:00:0000:0100:10m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 122.3 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.preprocessing import StandardScaler
from requests.auth import HTTPBasicAuth
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import numpy as np
import unidecode
import requests
import locale
import pickle
import re 
import lxml
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [77]:


driver = webdriver.Chrome()

try:
    # Load the page
    driver.get('https://www.senscritique.com/films/tops/top111')

    # Scroll down to load all content (you might need to adjust the range or use a more sophisticated scroll strategy)
    for _ in range(8):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # wait for content to load

    # Now the page should be fully loaded
    page_source = driver.page_source

finally:
    # Always close the driver to clean up
    driver.quit()

# Now you can use BeautifulSoup to parse the HTML
soup = BeautifulSoup(page_source, 'html.parser')

print(soup)


WebDriverException: Message: Can not connect to the Service /Users/zoe/.cache/selenium/chromedriver/mac-x64/118.0.5993.70/chromedriver


In [25]:


def film_info(link):

    # combine them to the new url
    base_url = 'https://www.senscritique.com'
    film_url = base_url + link

    # convert the url content to soup object
    film_page = requests.get(film_url)
    film_page_soup = BeautifulSoup(film_page.content, "html.parser")

    # use a new dict to save info
    extrainfo_dict = {}

    extrainfo_dict['film_year'] = film_page_soup.find('p', {'class': 'Text__SCTitle-sc-1aoldkr-1 CoverProductInfos__StyledText-sc-1un0kh1-13 eGhlHy jugtWW'}).text
    
    original_title_element = film_page_soup.find('p', {'class': 'Text__SCTitle-sc-1aoldkr-1 CoverProductInfos__StyledText-sc-1un0kh1-13 eGhlHy kuMSsq'})
    extrainfo_dict['original_title'] = original_title_element.text if original_title_element else None

    stats_elements = film_page_soup.select('p.Text__SCText-sc-1aoldkr-0.Stats__Text-sc-1u6v943-2.gATBvI')
    if len(stats_elements) == 3:
        extrainfo_dict['stars'] = stats_elements[0].text
        extrainfo_dict['saves'] = stats_elements[1].text
        extrainfo_dict['favorites'] = stats_elements[2].text
    
    critique_number_text = film_page_soup.select('.NavigationTab__WrapperTextStyled-sc-18dtd9d-7')[2].text
    extrainfo_dict['critique_number'] = int(critique_number_text.split('(')[-1].split(')')[0])

    return extrainfo_dict

In [31]:


count = 0


all_films_dict = {}
films = soup.find_all('div', class_="ProductListItem__Wrapper-sc-1jkxxpj-1 kusRkg")

for film in films:
        title = find_text(film, {'name': 'a', 'attrs': {"data-testid": "product-title"}})
        new_film_dict = {key: find_text(film, criteria) for key, criteria in search_criteria_dict.items()}

        link = film.find('a', {"data-testid": "product-title"})['href']

        new_film_dict.update(film_info(link))
        
        all_films_dict[title] = new_film_dict


print(all_films_dict)


{'Douze Hommes en colère (1957)': {'rating': '8.7', 'director': 'Sidney Lumet', 'duration': '1 h 36 min. ', 'genres': 'Policier, Drame', 'ranking': '1. Douze Hommes en colère (1957)', 'film_year': '1957', 'original_title': '12 Angry Men', 'stars': '51.7K', 'saves': '16.3K', 'favorites': '6.8K', 'critique_number': 592}, 'Harakiri (1962)': {'rating': '8.6', 'director': 'Masaki Kobayashi', 'duration': '2 h 13 min. ', 'genres': 'Drame', 'ranking': '2. Harakiri (1962)', 'film_year': '1962', 'original_title': 'Seppuku', 'stars': '8.2K', 'saves': '12.7K', 'favorites': '1.5K', 'critique_number': 114}, 'Blade Runner : The Final Cut (2007)': {'rating': '8.6', 'director': 'Ridley Scott', 'duration': '1 h 57 min. ', 'genres': 'Science-fiction', 'ranking': '3. Blade Runner : The Final Cut (2007)', 'film_year': '2007', 'original_title': None, 'stars': '1.2K', 'saves': '268', 'favorites': '111', 'critique_number': 3}, 'Le Bon, la Brute et le Truand (1966)': {'rating': '8.5', 'director': 'Sergio Leone

In [34]:
print(len(all_films_dict.keys()))

111


In [46]:
import json
with open('film_info.json', 'w') as json_file:
    json.dump(all_films_dict, json_file, indent=4)


In [47]:
df = pd.read_json('film_info.json', orient='index')

df.head()

,rating,director,duration,genres,ranking,film_year,original_title,stars,saves,favorites,critique_number
Douze Hommes en colère (1957),8.7,Sidney Lumet,1 h 36 min.,"Policier, Drame",1. Douze Hommes en colère (1957),1957,12 Angry Men,51.7K,16.3K,6.8K,592
Harakiri (1962),8.6,Masaki Kobayashi,2 h 13 min.,Drame,2. Harakiri (1962),1962,Seppuku,8.2K,12.7K,1.5K,114
Blade Runner : The Final Cut (2007),8.6,Ridley Scott,1 h 57 min.,Science-fiction,3. Blade Runner : The Final Cut (2007),2007,None,1.2K,268,111,3
"Le Bon, la Brute et le Truand (1966)",8.5,Sergio Leone,2 h 59 min.,"Western, Aventure","4. Le Bon, la Brute et le Truand (1966)",1966,"Il buono, il brutto, il cattivo",54.4K,13.6K,5.5K,379
Barberousse (1965),8.5,Akira Kurosawa,3 h 05 min.,Drame,5. Barberousse (1965),1965,Akahige,4.3K,7.7K,834,88


In [48]:
def convert_to_minutes(duration_str):
    parts = duration_str.split()
    hours = int(parts[0])
    minutes = int(parts[-2])
    total_minutes = hours * 60 + minutes
    return total_minutes

df['duration_minutes'] = df['duration'].apply(convert_to_minutes)


In [49]:
df.head(5)

,rating,director,duration,genres,ranking,film_year,original_title,stars,saves,favorites,critique_number,duration_minutes
Douze Hommes en colère (1957),8.7,Sidney Lumet,1 h 36 min.,"Policier, Drame",1. Douze Hommes en colère (1957),1957,12 Angry Men,51.7K,16.3K,6.8K,592,96
Harakiri (1962),8.6,Masaki Kobayashi,2 h 13 min.,Drame,2. Harakiri (1962),1962,Seppuku,8.2K,12.7K,1.5K,114,133
Blade Runner : The Final Cut (2007),8.6,Ridley Scott,1 h 57 min.,Science-fiction,3. Blade Runner : The Final Cut (2007),2007,None,1.2K,268,111,3,117
"Le Bon, la Brute et le Truand (1966)",8.5,Sergio Leone,2 h 59 min.,"Western, Aventure","4. Le Bon, la Brute et le Truand (1966)",1966,"Il buono, il brutto, il cattivo",54.4K,13.6K,5.5K,379,179
Barberousse (1965),8.5,Akira Kurosawa,3 h 05 min.,Drame,5. Barberousse (1965),1965,Akahige,4.3K,7.7K,834,88,185


In [74]:
df['ranking_number'] = df['ranking'].str.split('.').str.get(0)
#instead of using df['ranking'].str.split()[0]

df.head(5)

,rating,director,duration,genres,ranking,film_year,original_title,stars,saves,favorites,critique_number,duration_minutes,ranking_number,genres_split,stars_number,saves_number,favorites_number
Douze Hommes en colère (1957),8.7,Sidney Lumet,1 h 36 min.,"Policier, Drame",1. Douze Hommes en colère (1957),1957,12 Angry Men,51.7K,16.3K,6.8K,592,96,1,"[Policier, Drame]",51700.0,16300.0,6800.0
Harakiri (1962),8.6,Masaki Kobayashi,2 h 13 min.,Drame,2. Harakiri (1962),1962,Seppuku,8.2K,12.7K,1.5K,114,133,2,[Drame],8200.0,12700.0,1500.0
Blade Runner : The Final Cut (2007),8.6,Ridley Scott,1 h 57 min.,Science-fiction,3. Blade Runner : The Final Cut (2007),2007,None,1.2K,268,111,3,117,3,[Science-fiction],1200.0,268.0,111.0
"Le Bon, la Brute et le Truand (1966)",8.5,Sergio Leone,2 h 59 min.,"Western, Aventure","4. Le Bon, la Brute et le Truand (1966)",1966,"Il buono, il brutto, il cattivo",54.4K,13.6K,5.5K,379,179,4,"[Western, Aventure]",54400.0,13600.0,5500.0
Barberousse (1965),8.5,Akira Kurosawa,3 h 05 min.,Drame,5. Barberousse (1965),1965,Akahige,4.3K,7.7K,834,88,185,5,[Drame],4300.0,7700.0,834.0


In [52]:
df['genres_split'] = df['genres'].str.split(', ')
df_genre = df.explode('genres_split')

df_genre.head(5)

,rating,director,duration,genres,ranking,film_year,original_title,stars,saves,favorites,critique_number,duration_minutes,ranking_number,genres_split
Douze Hommes en colère (1957),8.7,Sidney Lumet,1 h 36 min.,"Policier, Drame",1. Douze Hommes en colère (1957),1957,12 Angry Men,51.7K,16.3K,6.8K,592,96,1.,Policier
Douze Hommes en colère (1957),8.7,Sidney Lumet,1 h 36 min.,"Policier, Drame",1. Douze Hommes en colère (1957),1957,12 Angry Men,51.7K,16.3K,6.8K,592,96,1.,Drame
Harakiri (1962),8.6,Masaki Kobayashi,2 h 13 min.,Drame,2. Harakiri (1962),1962,Seppuku,8.2K,12.7K,1.5K,114,133,2.,Drame
Blade Runner : The Final Cut (2007),8.6,Ridley Scott,1 h 57 min.,Science-fiction,3. Blade Runner : The Final Cut (2007),2007,None,1.2K,268,111,3,117,3.,Science-fiction
"Le Bon, la Brute et le Truand (1966)",8.5,Sergio Leone,2 h 59 min.,"Western, Aventure","4. Le Bon, la Brute et le Truand (1966)",1966,"Il buono, il brutto, il cattivo",54.4K,13.6K,5.5K,379,179,4.,Western


In [76]:
def deal_with_k(number_str):
    if 'K' in number_str:
        parts = number_str.split('K')[0]
        return float(parts) * 1000
    else:
        return int(number_str)

df['stars_number'] = df['stars'].apply(deal_with_k)
df['saves_number'] = df['saves'].apply(deal_with_k)
df['favorites_number'] = df['favorites'].apply(deal_with_k)

df.head(5)


,rating,director,duration,genres,ranking,film_year,original_title,stars,saves,favorites,critique_number,duration_minutes,ranking_number,genres_split,saves_number,favorites_number,stars_number
Douze Hommes en colère (1957),8.7,Sidney Lumet,1 h 36 min.,"Policier, Drame",1. Douze Hommes en colère (1957),1957,12 Angry Men,51.7K,16.3K,6.8K,592,96,1,"[Policier, Drame]",16300.0,6800.0,51700.0
Harakiri (1962),8.6,Masaki Kobayashi,2 h 13 min.,Drame,2. Harakiri (1962),1962,Seppuku,8.2K,12.7K,1.5K,114,133,2,[Drame],12700.0,1500.0,8200.0
Blade Runner : The Final Cut (2007),8.6,Ridley Scott,1 h 57 min.,Science-fiction,3. Blade Runner : The Final Cut (2007),2007,None,1.2K,268,111,3,117,3,[Science-fiction],268.0,111.0,1200.0
"Le Bon, la Brute et le Truand (1966)",8.5,Sergio Leone,2 h 59 min.,"Western, Aventure","4. Le Bon, la Brute et le Truand (1966)",1966,"Il buono, il brutto, il cattivo",54.4K,13.6K,5.5K,379,179,4,"[Western, Aventure]",13600.0,5500.0,54400.0
Barberousse (1965),8.5,Akira Kurosawa,3 h 05 min.,Drame,5. Barberousse (1965),1965,Akahige,4.3K,7.7K,834,88,185,5,[Drame],7700.0,834.0,4300.0


In [75]:
df = df.drop(columns=['stars_number'])
